<a href="https://colab.research.google.com/github/DanielHernandezGalvez/Jupiter_notebook/blob/main/metricas_descanso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import plotly.express as px

sheet_id = "1MdPYPcMC_Ujxbt7xr5qrsU_r46BdZEYV5gJop4_CrV0"
sheet_name = "Hoja1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)

# Asegurar que los nombres de las columnas están limpios
df.columns = df.columns.str.strip()

# Convertir la columna 'fecha' a formato datetime para graficar
df["fecha"] = pd.to_datetime(df["fecha"], format="%d/%m/%Y", errors="coerce")

# Función para convertir "hh:mm" a decimal (ejemplo: "5:30" → 5.5)
def hhmm_a_decimal(hhmm):
    try:
        horas, minutos = map(int, hhmm.split(":"))
        return horas + (minutos / 60)
    except ValueError:
        return float("nan")  # Si el formato es incorrecto, devolver NaN

# Aplicar la conversión en la columna "horas totales"
df["horas_totales"] = df["horas totales"].astype(str).apply(hhmm_a_decimal)

# Espaciador
def espacio(veces):
  print("\n" * veces)

# Eliminar filas con valores NaN en "horas_totales"
df = df.dropna(subset=["horas_totales"])

# Función para calcular horas faltantes de sueño por día
def horas_faltantes(dormidas):
    horas_necesarias = 8
    return max(horas_necesarias - dormidas, 0)

df["Horas Faltantes"] = df["horas_totales"].apply(horas_faltantes)

# Agregar columna de horas ideales (8h) para la gráfica
df["horas_ideales"] = 8

# Crear gráfico con Plotly
fig = px.line(df, x="fecha", y=["horas_totales", "horas_ideales"],
              labels={"value": "Horas de sueño", "fecha": "Fecha"},
              title="Comparación de Horas Dormidas vs Recomendadas",
              markers=True)

fig.update_traces(line=dict(width=2))
fig.show()

espacio(1)

In [5]:
import plotly.express as px

def calcular_sueño_semanal_plotly(df):
    df["fecha"] = pd.to_datetime(df["fecha"], format="%d/%m/%Y")
    df = df.sort_values(by="fecha", ascending=False).head(7)

    objetivo_semanal = 8 * 7
    horas_dormidas = round(df["horas_totales"].sum(), 2)
    horas_faltantes =round(max(objetivo_semanal - horas_dormidas, 0), 2)

    # Gráfico con Plotly
    df_plot = pd.DataFrame({"Categoría": ["Horas Dormidas", "Horas Faltantes"], "Horas": [horas_dormidas, horas_faltantes]})
    fig = px.bar(df_plot, x="Categoría", y="Horas", color="Categoría", title="Horas de Sueño Semanales",
                 color_discrete_map={"Horas Dormidas": "blue", "Horas Faltantes": "red"})
    fig.add_hline(y=56, line_dash="dash", line_color="gray", annotation_text="Objetivo (56h)")

    fig.show()

    return horas_faltantes

# Llamar a la función
horas_faltantes = calcular_sueño_semanal_plotly(df)
print(f"\nTe faltaron {horas_faltantes:.2f} horas de sueño esta semana.")


Te faltaron 15.92 horas de sueño esta semana.


In [6]:
# promedio de horas de sueñp
import pandas as pd

# Asegurar que la columna "minutos totales" tiene el formato correcto
if "minutos totales" in df.columns:
    df.rename(columns={"minutos totales": "minutos_totales"}, inplace=True)

# Convertir la columna de fechas a formato datetime si aún no lo está
df["fecha"] = pd.to_datetime(df["fecha"])

# Calcular el promedio histórico en horas
promedio_historico = (df["minutos_totales"] / 60).mean()

# Obtener la fecha de los últimos 7 días
ultima_fecha = df["fecha"].max()
df_ultimos_7_dias = df[df["fecha"] >= (ultima_fecha - pd.Timedelta(days=6))]

# Calcular el promedio de los últimos 7 días en horas
promedio_7_dias = (df_ultimos_7_dias["minutos_totales"] / 60).mean()

# Convertir a formato hh:mm
def convertir_a_horas_minutos(promedio):
    horas = int(promedio)
    minutos = int((promedio - horas) * 60)
    return f"{horas}:{minutos:02d} horas"

# Imprimir los resultados
print(f"Promedio histórico de sueño: {convertir_a_horas_minutos(promedio_historico)}")
print(f"Promedio de los últimos 7 días: {convertir_a_horas_minutos(promedio_7_dias)}")

Promedio histórico de sueño: 6:14 horas
Promedio de los últimos 7 días: 5:43 horas
